In [4]:
import sys
sys.path.append('..')

In [8]:
from importlib import reload
from os.path import exists

import networkx as nx
import numpy as np
import pandas as pd
from elasticsearch import Elasticsearch

from misc.obo import read_ontology
import misc.obo
import geo_annotation.search_utils
from geo_annotation.search_utils import search_ontology
from misc.utils import collapse_matches, zip_map_series
import pymongo
import pickle
import qgrid

reload(geo_annotation.search_utils)
reload(misc.obo)
reload(misc.utils)
pd.set_option('display.width', 512)

In [6]:
larisa_vd = pd.read_pickle('../data/larisa_set_uncleaned.pickle')
larisa_vd['series'] = larisa_vd['dataset_name'].str.split('_').str.get(0).str.split(' ').str.get(0)
larisa_vd['id'] = larisa_vd['series'].map(lambda s: int(s[3:]))
larisa_vd_ids = [int(x) for x in larisa_vd['id'].unique()]

es = Elasticsearch()
db = pymongo.MongoClient().scraper_meta
ontology = read_ontology('../data/geo-annotation/doid.obo',
                         exclude_duplicates=True)

Read obo graph
Name: 
Type: DiGraph
Number of nodes: 6595
Number of edges: 6594
Average in degree:   0.9998
Average out degree:   0.9998


In [12]:
larisa_vd_samples_ids_file = '../data/geo-annotation/larisa_vd_samples_ids.res1.pickle'
if not exists(larisa_vd_samples_ids_file):
    larisa_vd_samples_ids = [int(d['accession'][3:]) for d in db.samples.find({'series': {'$in': ['GSE{}'.format(i) for i in larisa_vd_ids]}},
                                        {'accession': 1})]
    with open(larisa_vd_samples_ids_file, 'wb') as f:
        pickle.dump(larisa_vd_samples_ids, f)
else:
    with open(larisa_vd_samples_ids_file, 'rb') as f:
        larisa_vd_samples_ids = pickle.load(f)
len(larisa_vd_samples_ids)

22524

In [15]:
annotation_result_file = '../data/geo-annotation/samples.disease.res1.pickle'
if not exists(annotation_result_file):
    res = search_ontology(client=es,
                          ontology=ontology,
                          index='samples',
                          ids=larisa_vd_samples_ids)

    print(len(res))

    import pickle

    with open(annotation_result_file, 'wb') as f:
        pickle.dump(res, f)

else:
    with open(annotation_result_file, 'rb') as f:
        res = pickle.load(f)

In [16]:
def doid_id(item_id):
    return "DOID:{:07}".format(item_id)


def ontology_name(item_id):
    return ontology.meta[item_id].name


def to_name(series_id):
    return "GSM{}".format(series_id)

In [18]:
view = pd.DataFrame.from_records(list(res.items()), columns=('id', 'matches'))
view['name'] = view['id'].map(to_name)
view['matches_names'] = view['matches'].map(lambda matches: [ontology_name(m) for m in matches])
view['matches_count'] = view['matches'].map(len)
view.set_index('name', inplace=True)

In [20]:
_t = view[view['matches_count'] == 1].matches_names.map(lambda x: x[0])
_t.value_counts()[:10]

disease                          1726
cancer                            630
breast cancer                     626
leukemia                          288
melanoma                          269
psoriasis                         257
hepatocellular carcinoma          245
endometriosis                     158
obesity                           130
brain glioblastoma multiforme      80
dtype: int64

In [36]:
view['collapsed_matches'] = view['matches'].map(collapse_matches(ontology.graph))
view['collapsed_matches_count'] = view['collapsed_matches'].map(len)
view['collapsed_matches_names'] = view['collapsed_matches'].map(lambda matches: [ontology_name(m) for m in matches])

qgrid.show_grid(view[['collapsed_matches', 'collapsed_matches_count', 'collapsed_matches_names']])

In [ ]:
# Очень много отсутсвующих путей по онтологии
# renal clear cell carcinoma, carcinoma
# sarcoma, peripheral primitive neuroectodermal...
# adenocarcinoma, colorectal adenocarcinoma

In [33]:
# Много мусора из description
_t = view[['collapsed_matches_names']]
_t = _t.collapsed_matches_names.apply(lambda x: pd.Series(x)).unstack().reset_index(level=0, drop=True)
_t.loc['GSM1540096']

name
GSM1540096                    adenocarcinoma
GSM1540096              large cell carcinoma
GSM1540096           squamous cell carcinoma
GSM1540096              small cell carcinoma
GSM1540096    tongue squamous cell carcinoma
GSM1540096     non-small cell lung carcinoma
GSM1540096         lung small cell carcinoma
GSM1540096                               NaN
GSM1540096                               NaN
dtype: object

In [35]:
# Вадафак! tongue squamous cell carcinoma == squamous-cell carcinoma ???
# такие гадости в онтологии можно отловить, если построить граф синонимов
# У таких нодов будут ребра из одного в другое и из другого в одно
ontology.meta['DOID:0050865'].dict()

{'alt_ids': [],
 'defs': ['A tongue cancer that is located_in the squamous cells of the tongue.'],
 'id': 'DOID:0050865',
 'is_a': 'DOID:8649',
 'is_obsolete': None,
 'name': 'tongue squamous cell carcinoma',
 'relationships': [],
 'subsets': [],
 'synonyms': ['squamous-cell carcinoma'],
 'xrefs': ['ICD10:C44', 'MSH:D002294']}

In [24]:
import qgrid
qgrid.show_grid(view[['collapsed_matches_names']])